In [4]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [3]:
!playwright install

128.3 Mb [                    ] 0% 0.0s128.3 Mb [                    ] 0% 31.4s128.3 Mb [                    ] 0% 26.3s128.3 Mb [                    ] 0% 20.0s128.3 Mb [                    ] 0% 18.2s128.3 Mb [                    ] 0% 15.6s128.3 Mb [                    ] 0% 15.4s128.3 Mb [                    ] 0% 13.0s128.3 Mb [                    ] 1% 11.8s128.3 Mb [                    ] 1% 10.9s128.3 Mb [                    ] 1% 10.2s128.3 Mb [                    ] 2% 9.4s128.3 Mb [                    ] 2% 8.7s128.3 Mb [=                   ] 2% 8.2s128.3 Mb [=                   ] 3% 8.0s128.3 Mb [=                   ] 3% 7.4s128.3 Mb [=                   ] 3% 7.1s128.3 Mb [=                   ] 4% 7.1s128.3 Mb [=                   ] 4% 6.7s128.3 Mb [=                   ] 5% 6.3s128.3 Mb [=                   ] 5% 6.0s128.3 Mb [=                   ] 6% 5.8s128.3 Mb [=                   ] 6% 5.7s128.3 Mb [=                   ] 7% 5.4s128.3 Mb [==                  ] 7% 5.4s128.3 Mb [==   